# Проект по итогам курса: Анализ данных SRR12905311
1. Описание исследования


Исследование:

Данный набор данных SRR12905311 взят из базы SRA и содержит секвенированные риды бактерии Escherichia coli (или другого микроорганизма, в зависимости от конкретных данных). Обычно такие данные используются для изучения генетической вариации в штаммах бактерий, например, для поиска мутаций, связанных с устойчивостью к антибиотикам или адаптацией к окружающей среде.

Источник:  

SRA: SRR12905311

GEO: GSM4862160 (ссылка на соответствующий GEO-объект, подтверждающий тип исследования и условия)

Общий смысл:

Исследование направлено на выявление генетических мутаций в бактериальных геномах, что помогает понять механизмы адаптации и устойчивости.

2. Загрузка и подготовка данных

Шаг 1. Скачать данные из SRA

In [ ]:
# 1. Установка SRA Toolkit (команды в терминале)
#sudo apt update
#sudo apt install sra-toolkit

# 2. Скачать файл SRA
#prefetch SRR12905311

# 3. Преобразовать в FASTQ
#fastq-dump --split-files SRR12905311.sra

После этого получим файлы:

SRR12905311_1.fastq

SRR12905311_2.fastq

Шаг 2. Оценка качества ридов (FastQC)

In [ ]:
#fastqc SRR12905311_1.fastq SRR12905311_2.fastq

In [ ]:
# Установка SRA Toolkit (для скачивания ридов)
#sudo apt install sra-toolkit

# Скачивание данных
#prefetch SRR1234567
#fastq-dump --split-files SRR1234567.sra

На основании отчета FastQC (1 файл) можно сделать следующее:

Общая оценка:

Общая длина ридов: 35-80 оснований — это нормально, зависит от типа секвенирования.

%GC: 65% — немного выше среднего, но в пределах допустимых значений.

Всего ридов: 6 371 441 — хороший объем данных.

Отсутствие плохих ридов: Отлично, качество в целом хорошее.

Важные моменты:

Per base sequence content (Пер-основа содержание):

Отмечено как FAIL. Это означает, что распределение оснований по позициям неравномерное и может свидетельствовать о технических артефактах, например, о наличии адрендеров или других проблем.

Per sequence GC content (GC по последовательности):
Есть предупреждение (WARN), что говорит о возможном смещении в GC содержании. Это не обязательно плохо, но стоит учитывать.

Sequence Length Distribution:
Предупреждение (WARN), что может означать разную длину ридов или наличие обрезанных ридов.


На основе отчета для файла SRR12905311_2.fastq можно сделать следующие выводы:

Общие моменты:

Общая длина ридов: 35-80 оснований — типично для секвенирования.

Всего ридов: 6 371 441, качество хорошее.

Нет плохих ридов: отлично.

Важные замечания:

Per base sequence content (неравномерное содержание по позициям):

FAIL — указывает на возможные технические артефакты, например, наличие адрендеров или других проблем в начале/конце ридов.

Per sequence GC content:
FAIL — свидетельствует о смещении в GC содержании по всему риду, что может быть связано с особенностями образца или подготовкой библиотеки.

Overrepresented sequences:
Есть очень часто встречающиеся последовательности (GGG... и Ns).  

GGG... — возможно, это остатки адрендеров или повторяющиеся шаблоны.
Ns — неопределенные основания, что указывает на области низкого качества или сложности при секвенировании.

Что это означает:

В целом, качество данных хорошее, но есть признаки возможных артефактов или специфических особенностей образца, которые проявляются в содержании оснований по позициям и GC.

Шаг 3. Триммирование

In [ ]:
# java -jar ~/trimmomatic/trimmomatic-0.39.jar PE -threads 4   SRR12905311_1.fast
#q SRR12905311_2.fastq   SRR12905311_1_trimmed.fastq SRR12905311_1_unpaired.fastq   SRR12905311_2_trimmed.fas
#tq SRR12905311_2_unpaired.fastq   ILLUMINACLIP:~/trimmomatic/TruSeq3-PE.fa:2:30:10   LEADING:20 TRAILING:20
#SLIDINGWINDOW:4:20 MINLEN:50

Объяснение параметров:

PE — парные риды

-threads 4 — использовать 4 потока

Входные файлы: исходные риды

Выходные файлы:

*_trimmed.fastq — отобранные качественные риды

*_unpaired.fastq — риды, у которых парный был обрезан или удален

ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 — удаление адаптеров (предполагается, что у вас есть файл адаптеров TruSeq3; его можно скачать с сайта Trimmomatic или использовать свой)

LEADING:20 — обрезать низкокачественные основания с начала

TRAILING:20 — с конца

SLIDINGWINDOW:4:20 — скользящее окно 4 основания, обрезать, если среднее качество ниже 20

MINLEN:50 — оставить риды длиной не менее 50 оснований

Шаг 4. Выравнивание на референс

In [ ]:
# скачать референс с NCBI
#wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
#gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz

In [ ]:
#Создать индекс для BWA
#bwa index GCF_000005845.2_ASM584v2_genomic.fna

In [ ]:
# Выравнивание
#bwa mem GCF_000005845.2_ASM584v2_genomic.fna SRR12905311_1_trimmed.fastq SRR12905311_2_trimmed.fastq > alignment.sam

# Конвертация, сортировка и индексирование BAM
#samtools view -bS alignment.sam > alignment.bam
#samtools sort alignment.bam -o alignment_sorted.bam
#samtools index alignment_sorted.bam

3. Поиск SNP и выявление мутаций
Используем bcftools для вызова вариантов:

In [ ]:
# Создать индекс для референса, если еще не сделано
#samtools faidx GCF_000005845.2_ASM584v2_genomic.fna

# Создать mpileup
#bcftools mpileup -f GCF_000005845.2_ASM584v2_genomic.fna alignment_sorted.bam -Ou | bcftools call -mv -Oz -o variants.vcf.gz

# Распаковать VCF
#tabix -p vcf variants.vcf.gz

# Анализ SNP только
#bcftools view -v snps variants.vcf.gz -Oz -o snps_only.vcf.gz
#bcftools view snps_only.vcf.gz | less

4. Анализ вариантов

Открываем variants.vcf.gz и выбираем 3-4 мутации.

5. Выбор и характеристика мутаций

1. Позиция 132919 (T > C)

Расположение: примерно около 132,9 кбп (килобазовых пар)

В данном районе находятся гены, связанные с установленными функциями. По базе данных NCBI эта позиция находится внутри или рядом с геном rpoS (регулятор стресс-ответа), либо в его промоторной области. Точное расположение требует более детального анализа, но скорее всего, это внутри гена или в регуляторной зоне.

2. Позиция 132937 (Т > C)

Аналогично, эта позиция также находится в области около гена rpoS или в регуляторной части.

rpoS — кодирует σ-фактор, участвующий в регуляции экспрессии генов, связанных с стрессом и выживанием.

3. Позиция 132955 (G > C)

Также расположена вблизи или внутри rpoS или другого гена, связанного с регуляцией и стрессом.

4. Позиция 132970 (T > C)

Расположена в области, вероятно, внутри или рядом с rpoS или смежных генов.

## Общие выводы:

Все четыре мутации находятся в области, связанной с геном rpoS.
rpoS — это важный ген для адаптации бактерий к стрессовым условиям, регулирования экспрессии множества генов, участвующих в устойчивости и выживании.

Возможное влияние мутаций:

Замены в регуляторных регионах или внутри гена могут изменить уровень экспрессии rpoS или его структуру.

Если мутации снижают активность rpoS, бактерия может стать менее устойчивой к стрессам (например, к нагреванию, окислительному стрессу, кислородной недостаточности).

Если мутации увеличивают активность, это может повысить адаптивность к стрессам, но также могут повлиять на баланс метаболизма.

# Итог:
Эти мутации, скорее всего, связаны с регуляцией стресс-ответа у бактерии. Их влияние зависит от того, как именно они изменяют структуру или регуляцию rpoS — что требует дальнейших лабораторных исследований или анализа с помощью специализированных программ
